# Baseline: Matrix Factorization with InfoNCE

**NOTE: This notebook demonstrates the legacy MLP-based Two-Tower model. For the state-of-the-art Sequential Transformer, see `sequential_transformer.ipynb`.**

This is an easy-to-train baseline (bi-encoder).

`Users -> Embedding` | `Items -> Embedding`  -> Cosine Similarity -> InfoNCE Loss

*   **Architecture:** Shallow MLP (or just embeddings). No deep sequence modeling.
*   **Training Time:** ~5 minutes on GPU.
*   **Performance:** ~50% improvement over NCF (Neural Collaborative Filtering), but significantly lower than the Sequential Transformer.

### Protocol
*   **Data Split:** Leave-One-Out (LOO).
*   **Positive Samples:** All interactions treated as implicit feedback.
*   **Metrics:** HR@10, NDCG@10 (Sampled Evaluation against 100 negatives + 1 positive).


### Setup (Colab)
Run the following cell to install the package and dependencies if running in Google Colab.
If running locally, ensure you have installed the package via `pip install -e .`

In [ ]:
!git clone https://github.com/zheliu17/nanoRecSys.git
%pip install -q -e ./nanoRecSys

import psutil  # noqa: F401

# In fact, we don't need psutil. force-reinstall to trigger colab restart
%pip install --force-reinstall psutil=={psutil.__version__}
print("Installation complete. Please restart runtime...")

In [ ]:
import nanoRecSys.data.build_dataset
import nanoRecSys.data.splits

nanoRecSys.data.build_dataset.process_data()
nanoRecSys.data.splits.create_user_time_split()

In [ ]:
import nanoRecSys.train


class Args:
    mode = "retriever"
    user_tower_type = "mlp"

    epochs = 3
    check_val_every_n_epoch = 1
    batch_size = 4096
    adam_beta2 = 0.999

    lr = 0.05
    num_workers = 2


nanoRecSys.train.main(Args)

Evaluate the popularity baseline:


Note: 
- Sampled evaluation, not full ranking. Number below are comparable to https://arxiv.org/abs/1904.06690
- MRR in paper is global MRR, and should be compared to MRR@100 here.

In [ ]:
from nanoRecSys.eval.offline_eval import OfflineEvaluator

evaluator = OfflineEvaluator(1024, sampled=True, sample_strategy="popularity")
results = evaluator.eval_popularity()

df = evaluator.formatted_results(results)
df

Evaluate our model

In [ ]:
results = evaluator.eval_retrieval()
df = evaluator.formatted_results(results)
df

Results:

|        | HR@10  | NDCG@10 | MRR (Global)   |
|--------|--------|---------|--------|
| Popularity| 0.1426 |  0.0717 | 0.0722 |
| NeuCF    | 0.2922 | 0.1271  | 0.1072 |
| **Ours**   | **0.4392** |  **0.232** | **0.1916** |